In [1]:
import chess
import chess.uci
import chess.pgn
import os
import csv
import matplotlib.pyplot as plt
from numpy import array
import numpy as np
import math
import time
from multiprocessing import Pool
import psutil


In [2]:


# writing_file = open(writing_path, "a")
# writer = csv.DictWriter(writing_file, fieldnames=csv_columns)
#writer = csv.writer(myfile)

#stockfish_performances = open(writing_path, 'a')


In [3]:
def write_dict_to_csv(csv_file, csv_columns, dict):
    with open(csv_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        #writer.writeheader()
        #for data in dict_data:
        writer.writerow(dict)
    return 

In [4]:
handler = chess.uci.InfoHandler()
engine = chess.uci.popen_engine("/Applications/Stockfish/src/stockfish")
engine.info_handlers.append(handler)

In [5]:
EVALTIME = 1000 #ms


In [6]:
def check_elos(game):
    print("white elo: " + game.headers["WhiteElo"])
    print("black elo: " + game.headers["BlackElo"])

In [7]:
def get_performance_dicts(game): 
    handler = chess.uci.InfoHandler()
    engine = chess.uci.popen_engine("/Applications/Stockfish/src/stockfish")
    engine.info_handlers.append(handler)
    
    board = game.board()

    score = 0
    prev_score = 0
    wp = {}
    bp = {}
    evals = []
    
    
    #wp.append(game.headers["WhiteElo"])
    #bp.append(game.headers["BlackElo"])
    wp["elo"] = game.headers["WhiteElo"]
    wp["opp_elo"] = game.headers["BlackElo"]

    bp["elo"] = game.headers["BlackElo"]
    bp["opp_elo"] = game.headers["WhiteElo"]
    
    
    result = game.headers["Result"]
    r1, r2 = interpret_result(result)
    
#     wp.append(r1)
#     bp.append(r2)
    wp["result"] = r1
    bp["result"] = r2
    
#     wp.append(game.headers["ECO"])
#     bp.append(game.headers["ECO"])

    wp["opening"] = game.headers["ECO"]
    bp["opening"] = game.headers["ECO"]

    wp["eval_time"] = EVALTIME/1000
    bp["eval_time"] = EVALTIME/1000
 
    
    evaluation = engine.go(movetime=EVALTIME)
    #time.sleep(EVALTIME/1000)
    score = handler.info["score"][1].cp
    evals.append(score)

    move_count = 0
    prev_score = score
    
#     print("Beginning game with score: " + str(prev_score))
    
    wp_moves = []
    bp_moves = []
    
    for move in game.main_line():
        board.push(move)
        engine.position(board)
        move_count += 1
        evaluation = engine.go(movetime=EVALTIME)
        #time.sleep(EVALTIME/1000)

        score = handler.info["score"][1].cp
        if score is None:
#             print("Finishing game...")
            break
        if move_count % 2 == 1:
            score = -score
        evals.append(score)
        dif = score - prev_score
        # print("dif calculated as: " + str(score) + " minus " + str(prev_score))
        # performance.append("%.2f" % dif)
        if move_count % 2 == 1:
            wp_moves.append("%.2f" % dif)
        elif move_count % 2 == 0:
            bp_moves.append("%.2f" % -dif)
        prev_score = score

    wp["move_evals"] = wp_moves
    wp["opp_move_evals"] = bp_moves
    
    bp["move_evals"] = bp_moves
    bp["opp_move_evals"] = wp_moves

#     print(wp)
#     print(bp)
    return wp, bp

In [8]:
def interpret_result(result):
    r1 = -1.0
    r2 = -1.0
    if len(result) == 3:
        r1 = float(result[0])
        r2 = float(result[2])
    if len(result) == 7:
        r1 = 0.5
        r2 = 0.5
    assert r1 != -1.0, "Bad result detected"
    assert r2 != -1.0, "Bad result detected"

    return r1, r2

In [9]:
def skip_to_game(pgn_file, game_num):

    offsets = chess.pgn.scan_offsets(pgn_file)
    for i in range(game_num):
        book_mark = next(offsets)
        if i == (game_num - 1):
            pgn_file.seek(book_mark)
    #return pgn_file
# first_game_offset = next(offsets)
# second_game_offset = next(offsets)






In [10]:
def evaluate_game(game):
#     starting_position = 0
#     with open(pgns_all_path) as pgns:
#         skip_to_game(pgns, starting_position + i)
    
#     for i in range(amount):

#         #pgns = open(pgns_all_path)

#         game = chess.pgn.read_game(pgns)
    performances = get_performance_dicts(game)
    for performance_dict in performances:
        write_dict_to_csv(writing_file, csv_columns, performance_dict)

    #return p1, p2
#         for p in performances:
#             #with open('dict.csv', 'wb') as csv_file:
#             for key, value in performances:
#                 WriteDictToCSV(writing_file, csv_columns, dict_data)
#                 stockfish_performances.writerow([key, value])
#         #stockfish_performances.write(str(p) + "\n")
#         stockfish_performances.flush()
#         print("Finished a game")


In [ ]:
writing_file = "/Users/tylerahlstrom/Documents/GitHub/DI_Proposal/data/stockfish_performances_onesecond.csv"
csv_columns = ['elo','opp_elo','opening', 'result', 'eval_time', 'move_evals', 'opp_move_evals']

pgns_all_path = "/Users/tylerahlstrom/Documents/GitHub/DI_Proposal/data/lichess_db_standard_rated_2017-03.pgn"
pgns = open(pgns_all_path)

progress_count = 0
starting_game = 0
batch_size = 20
num_batches = 500

with open(writing_file, 'a') as csvfile:
    if os.stat(writing_file).st_size == 0:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()

if starting_game > 0:
    skip_to_game(pgns, starting_game)

if __name__ == '__main__':
    
    
    for i in range (num_batches):
        raw_games = []
        for j in range(batch_size):
            pgn = chess.pgn.read_game(pgns)
            raw_games.append(pgn)
        pool = Pool(8)
        pool.map(evaluate_game,raw_games)
        progress_count += batch_size * 2
        print("Performances evaluated: " + str(progress_count))
        os.system('pkill stockfish')

Performances evaluated: 40
Performances evaluated: 80
Performances evaluated: 120
Performances evaluated: 160
Performances evaluated: 200
Performances evaluated: 240
Performances evaluated: 280
Performances evaluated: 320
Performances evaluated: 360
